# Import Libraries

In [ ]:
import copy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

# Generate 2-D data with 3 clusters

In [ ]:
np.random.seed(123) #Set seed for replicability

#Create three clusters of points drawn from a multivariate normal distribution
mean1=[0, 0]
cov1 = [[1, .1], [.1, 1]]
x1, y1 = np.random.multivariate_normal(mean1, cov1, 100).T

mean2=[4.5, 1.5]
cov2 = [[1, .5], [.5, 1]]
x2, y2 = np.random.multivariate_normal(mean2, cov2, 100).T

mean3=[4, -3]
cov3 = [[1, .2], [.2, 1]]
x3, y3 = np.random.multivariate_normal(mean3, cov3, 100).T

#Combine the three clusters into one dataset
x=np.hstack([x1,x2,x3])
y=np.hstack([y1,y2,y3])

#Visualize the data
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(x, y,c='k',s=5)
plt.title("Simulated Data")
plt.show()

# K-means

The idea is to find $K$ groups of observations (clusters), denoted by $C_k$, which are similar to one another. The mathematical objective is to partition observations into $K$ sets so as to minimize the within-cluster sum of squares:

$$ Minimize \displaystyle \sum_{k=1}^K \sum_{\mathrm{x}_n \in C_k} ||\mathrm{x}_n - \mu_k ||^2 with \ respect \ to \ \displaystyle C_k, \ \mu_k$$

where $\mu_k$ is the mean point of $C_k$, and is referred to as *centroid*.

## Approach: Iterative Refinement (Lloyd's algorithm)

- Step 0: Start with an initial guess of a set of centroids $\mu_k$.
- Step 1: Create clusters containing points closest in distance to each centroid
- Step 2: Update the centroids as the means of all points in each cluster.
- Step 3: Repeat 1 and 2 until the assignments of clusters and centroids does not change (or max number of steps reached)

## Step 0: Start with the initial guess (picked at random) 

In [ ]:
K=4 #Consider 4 clusters

#Step 0: Initial Guess
np.random.seed(121) #Set seed for replicability
mu0= [ np.array((np.random.uniform(x.min(),x.max()), \
       np.random.uniform(y.min(),y.max()))) for i in range(K)] 
  
print("Initial Guess:")
print(np.round(mu0,2))

In [ ]:
#Visualize the data and initial guesses
colors=['red','green','blue','cyan']
fig, ax = plt.subplots(figsize=(6,4))
for i,m in enumerate(mu0):
    plt.plot(m[0],m[1],'X',label="initial guess for centroid "+str(i+1),c=colors[i])
ax.scatter(x, y, c='k',s=5)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

## Step 1: Create clusters containing points closest in distance to each centroid

In [ ]:
clusters=np.empty(x.shape,dtype=int) #allocate space to keep track of the cluster each point is in

In [ ]:
for i,p in enumerate(zip(x,y)): #Loop over all points and determine which centroid is the closest
    d=np.array([np.linalg.norm(p-mu0[k]) for k in range(K)])
    bestKindex=np.argmin(d)
    clusters[i]=bestKindex

In [ ]:
clusters

In [ ]:
#Visualize the closest points
colors=['red','green','blue','cyan']
clrs=[colors[i] for i in clusters]
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(x, y,c=clrs,s=5)
for i,m in enumerate(mu0):
    plt.plot(m[0],m[1],'X',label="initial guess for centroid "+str(i+1),c=colors[i])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

# Step 2: Update the centroids as the means of all points in each cluster.

In [ ]:
mu1= [np.array((np.mean(x[clusters==k]),\
                np.mean(y[clusters==k]))) for k in range(K)] 
print("Updated Guess")
np.round(mu1,2)

In [ ]:
#Visualize the updated guesses
colors=['red','green','blue','cyan']
clrs=[colors[i] for i in clusters]
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(x, y,c=clrs,s=5)
for i,m in enumerate(mu1):
    plt.plot(m[0],m[1],'X',label="initial guess for centroid "+str(i+1),c=colors[i])
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

# Step 3: Iterate

In [ ]:
diff=sum([np.linalg.norm(mu1[k]-mu0[k]) for k in range(K)])
n=2
nmax=100

while diff>.0001 and n<nmax:
    print("------------------")
    print("Iteration:",n)
    n+=1
    
    mu0=mu1
    
    for i,p in enumerate(zip(x,y)):
        
        d=np.array([np.linalg.norm(p-mu0[k]) for k in range(K)])
        bestKindex=np.argmin(d)
        clusters[i]=bestKindex
    
    mu1= [np.array((np.mean(x[clusters==k]),\
                    np.mean(y[clusters==k]))) for k in range(K)] 
    
    print("Updated Guess:")
    print(np.round(mu1,2))
    diff=sum([np.linalg.norm(mu1[k]-mu0[k]) for k in range(K)])
    print("diff=",diff)
    


In [ ]:
print("Cluster Centers:", mu1)
print(clusters)

In [ ]:
colors=['red','green','blue','cyan']
clrs=[colors[i] for i in clusters]
fig, ax = plt.subplots(figsize=(6,4))
ax.scatter(x, y,c=clrs,s=5)
for i,m in enumerate(mu0):
    plt.plot(m[0],m[1],'X',label="final guess for centroid "+str(i+1),c=colors[i])
    
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

In [ ]:
#Using other libraries
from sklearn.cluster import KMeans

data=pd.DataFrame({"x":x,"y":y}) #combine data into a data frame

kmeans=KMeans(n_clusters=4).fit(data[['x','y']])
print("Labels:",kmeans.labels_)
print("Centers:",kmeans.cluster_centers_)
print("Inertia:",kmeans.inertia_) #Proxy for SSE
data['Cluster']=kmeans.labels_

In [ ]:
kmeans=KMeans(n_clusters=3).fit(data[['x','y']])
print("Labels:",kmeans.labels_)
print("Centers:",kmeans.cluster_centers_)
print("Inertia:",kmeans.inertia_) #Proxy for SSE
data['Cluster']=kmeans.labels_

In [ ]:
kmeans=KMeans(n_clusters=4).fit(data[['x','y']])
print("Labels:",kmeans.labels_)
print("Centers:",kmeans.cluster_centers_)
print("Inertia:",kmeans.inertia_) #Proxy for SSE
data['Cluster']=kmeans.labels_

In [ ]:
#Determining number of clusters
nClusters=range(2,10)
sumDistances=[]
for n in nClusters:
    kmeans=KMeans(n_clusters=n).fit(data)
    sumDistances.append(kmeans.inertia_) #Proxy for SSE

In [ ]:
plt.plot(nClusters,sumDistances,'-')
plt.xlabel('nClusters')
plt.ylabel('Sum Of Distances')
plt.show()

In [ ]:
kmeans=KMeans(n_clusters=4).fit(data[['x','y']])
print(kmeans.cluster_centers_)